In [26]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import os


In [27]:
num_epochs=10
batch_size=100
lr=0.001

train_dataset = dsets.MNIST(root="./data",
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)


test_dataset = dsets.MNIST(root="./data",
                           train=False,
                           transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [30]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.layer1 = nn.Sequential(                    #input size (1,28,28)
            nn.Conv2d(1, 16, kernel_size=5, padding=2), #in_channel=1,out_channel=16,filter size=5
            nn.BatchNorm2d(16),     # o/p size (16,28,28)
            nn.ReLU(),              #activation  o/p size  (16,28,28)
            nn.MaxPool2d(2))        #output size after pooling (16,14,14)
        
        
        self.layer2 = nn.Sequential(    #input shape (16,14,14)
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))          #output shape (32,7,7)
        
        self.fc = nn.Linear(32*7*7,10)     # fully connected Layer,output 10 classes
    
    def forward(self,x):
        
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0),-1)    #flatten the output of conv2d to feed into fully connected layers
        
        out = self.fc(out)
        return out   


cnn = CNN()
# print(cnn)

In [37]:
if os.path.isfile("pkl/cnn.pkl"):
    cnn.load_state_dict(torch.load("pkl/cnn.pkl"))


else:
    criterian = nn.CrossEntropyLoss() #LOSS
    optimizer = torch.optim.Adam(cnn.parameters(),lr=lr)
    
    
    for epoch in range(num_epochs):
        for i, (images,labels) in enumerate(train_loader):   # gives batch data ,normalize x when iterate train_loader
            
            images = Variable(images)
            labels = Variable(labels)
            
            optimizer.zero_grad()    #clear gradients for this training step
            
            outputs=cnn(images)     #cnn  output
            
            loss=criterian(outputs,labels)  #cross entropy loss
            
            loss.backward()      #backpropagation ,compute gradients
            
            optimizer.step()     #apply gradients
            
            if (i+1) % 100 ==0:
                print("Epoch [%d / %d] ,iter [%d / %d] Loss:%.4f"
                      %(epoch+1,num_epochs,i+1,len(train_dataset)//batch_size,loss.data))
                if not os.path.isfile("pkl/cnn.pkl"):
                    torch.save(cnn.state_dict(),"pkl/cnn.pkl")
                
                

Epoch [1 / 10] ,iter [100 / 600] Loss:0.0587
Epoch [1 / 10] ,iter [200 / 600] Loss:0.0188
Epoch [1 / 10] ,iter [300 / 600] Loss:0.0397
Epoch [1 / 10] ,iter [400 / 600] Loss:0.0265
Epoch [1 / 10] ,iter [500 / 600] Loss:0.0430
Epoch [1 / 10] ,iter [600 / 600] Loss:0.0233
Epoch [2 / 10] ,iter [100 / 600] Loss:0.0527
Epoch [2 / 10] ,iter [200 / 600] Loss:0.0100
Epoch [2 / 10] ,iter [300 / 600] Loss:0.0228
Epoch [2 / 10] ,iter [400 / 600] Loss:0.0028
Epoch [2 / 10] ,iter [500 / 600] Loss:0.0032
Epoch [2 / 10] ,iter [600 / 600] Loss:0.0146
Epoch [3 / 10] ,iter [100 / 600] Loss:0.0288
Epoch [3 / 10] ,iter [200 / 600] Loss:0.0117
Epoch [3 / 10] ,iter [300 / 600] Loss:0.0017
Epoch [3 / 10] ,iter [400 / 600] Loss:0.0366
Epoch [3 / 10] ,iter [500 / 600] Loss:0.0188
Epoch [3 / 10] ,iter [600 / 600] Loss:0.0180
Epoch [4 / 10] ,iter [100 / 600] Loss:0.0070
Epoch [4 / 10] ,iter [200 / 600] Loss:0.0336
Epoch [4 / 10] ,iter [300 / 600] Loss:0.0226
Epoch [4 / 10] ,iter [400 / 600] Loss:0.0070
Epoch [4 /

In [38]:
cnn.eval()
correct = 0
total = 0

for images ,labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    
    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted ==labels).sum()
    
print("Test Accuracy 10000 test images = %f %%" %(100*correct/total))

Test Accuracy 10000 test images = 99.129997 %
